<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/chat_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re

In [5]:
import pandas as pd

In [6]:
import numpy as np
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 5.9 MB/s eta 0:00:00


In [7]:
import emoji

In [8]:
from collections import Counter

In [9]:
import matplotlib.pyplot as plt

In [10]:
from PIL import Image


In [11]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [27]:
# extract time
def date_time(s):
 pattern='^(0?[1-9]|1[0-2])\/(0?[1-9]|[12][0-9]|3[01])\/([0-9]{2}|[0-9]{4})$'
 result=re.match(pattern,s)
 if result:
  return True
 else:
  return False

<>:3: SyntaxWarning: invalid escape sequence '\/'
<>:3: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipython-input-4019334591.py:3: SyntaxWarning: invalid escape sequence '\/'
  pattern='^(0?[1-9]|1[0-2])\/(0?[1-9]|[12][0-9]|3[01])\/([0-9]{2}|[0-9]{4})$'


In [13]:
# find author and contact
def author_contact(s):
  s=s.split(":")
  if len(s)==2:
    return True
  else:
      return False

In [25]:
import re

# find the message
def datapoint(line):
  splitline=line.split(' - ', 1) # Split only on the first occurrence of ' - '
  dateTime=splitline[0]
  date,time=dateTime.split(',', 1) # Split date and time
  message=" ".join(splitline[1:])

  # Ensure author_contact is defined or use directly
  # Assuming author_contact is available in the notebook
  if author_contact(message):
    splitmessage=message.split(": ", 1) # Split on ': ' for author and message
    author=splitmessage[0]
    message=" ".join(splitmessage[1:])
  else:
    author= None
  return date.strip(),time.strip(),author,message.strip()

In [15]:
# it doesn’t matter if you are using a group chat dataset or your conversation with one person. All the functions defined above will prepare your data for the task of sentiment analysis as well as for any data science task. Now here is how we can prepare the data we collected from WhatsApp by using the above functions:



In [28]:
data=[]
conversation='Whatsaap chat with rasti.txt'
with open(conversation,encoding='utf-8') as fp:
  fp.readline() # Discard the first line (often chat export header)
  messageBuffer=[]
  current_date, current_time, current_author = None, None, None

  while True:
    line=fp.readline()
    if not line:
      break # End of file

    line=line.strip()
    if not line: # Skip empty lines
      continue

    # Check if the line starts with a date pattern, indicating a new message
    # Example: '12/25/2023, 10:30 AM - Author: Message'
    # We need to extract '12/25/2023' to check with date_time function
    parts = line.split(' - ', 1)
    if len(parts) > 0: # Ensure there's a date-time part
        dt_part = parts[0]
        date_part_of_line = dt_part.split(',', 1)[0].strip() # Get '12/25/2023'

        if date_time(date_part_of_line):
            # This is a new message line
            if current_author is not None: # If we have a previous message buffered, save it
                data.append([current_date, current_time, current_author, ' '.join(messageBuffer)])
                messageBuffer.clear()

            # Process the current line as a new message
            current_date, current_time, current_author, message_part = datapoint(line)
            messageBuffer.append(message_part)
        else:
            # This is a continuation of the previous message
            if messageBuffer: # Only append if there's an ongoing message
                messageBuffer.append(line)
            else:
                # This case might happen if the file doesn't start with a valid date line
                # or if a non-date line appears before any date line.
                # For now, let's just append it to the buffer and hope a date line follows or previous message exists.
                messageBuffer.append(line)

  # After the loop, append any remaining message in the buffer
  if current_author is not None and len(messageBuffer) > 0:
    data.append([current_date, current_time, current_author, ' '.join(messageBuffer)])

In [17]:
#  now here we can analyze the whatsaap chat message

In [29]:
df=pd.DataFrame(data,columns=["Date",'Time','Author','Message'])
df['Date']=pd.to_datetime(df['Date'])

/tmp/ipython-input-1815550880.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date']=pd.to_datetime(df['Date'])


In [30]:
data = df.dropna()

In [33]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment= SentimentIntensityAnalyzer()
data['Positive']=[sentiment.polarity_scores(i)["pos"] for i in data ["Message"]]
data['Negative']=[sentiment.polarity_scores(i)["neg"]for i in data ["Message"]]
data['Neutal']=[sentiment.polarity_scores(i) for i in data ["Message"]]
print(data.head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


        Date      Time                     Author  \
0 2026-02-04   9:32 AM   Rasti Sih Section B sahu   
1 2026-02-04  10:05 AM  Rasti Sih Section B sahu:   
2 2026-02-04   9:37 AM   Rasti Sih Section B sahu   
3 2026-02-04  10:22 AM       SHIVA PRAJAPATI ---⚡   
4 2026-02-04  11:24 AM   Rasti Sih Section B sahu   

                                  Message  Positive  Negative  \
0                                     Hmm       0.0       0.0   
1                                               0.0       0.0   
2  Yar kuch nhi padh rhi kha se start kru       0.0       0.0   
3                               Unit 1 se       0.0       0.0   
4                         <Media omitted>       0.0       0.0   

                                              Neutal  
0  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...  
1  {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...  
2  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...  
3  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...  
4  {'neg': 0.0,

In [36]:
x=sum(data['Positive'])
y=sum(data['Negative'])
z=sum(item['neu'] for item in data['Neutal'])

def sentiment_score(a,b,c):
  if (a>b) and (a>c):
    print("positive 😊")
  elif (b>a) and (b>c):
    print("Negative 😠")
  else:
    print("Neutal 🙂")
sentiment_score(x,y,z)

Neutal 🙂


In [ ]:
#  so ,the data is used to indicates that most of  the message between me and other person are neutral.which means it is neither positive or netural